# TESTE DE WEB SCRAPPING

Endereço web: http://www.imoveisnocentrosp.com.br/buscar?venda=Sim

capturar os seguintes campos em uma primeira rodada.

- Nome do imóvel 
- Valor
- Descrição
- Quantidade de quartos
- Quantidade de banheiros
- Quantidade de vagas
- Area em m2
- Código do imóvel

Depois desta primeira ação paginar por um determinado número de páginas (dado pelo usuário)

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
from time import sleep
from random import randint

# Determinando a URL

In [2]:
url="http://www.imoveisnocentrosp.com.br/buscar?venda=Sim"

### Capturando a URL com GET

In [3]:
get(url)

<Response [200]>

In [4]:
conteudo = get(url).content

### Transformando o conteúdo em HTML

In [5]:
conteudo = BeautifulSoup(conteudo,'html.parser')

# Identificando a posição inicial dos itens

In [6]:
imoveis_centro = conteudo.find('div', class_='tm-property__wrap').find_all('div',  class_='tm-property__item')

### Verificando quantos itens coletados

In [7]:
print(len(imoveis_centro))

15


# Testando as posições dos itens a serem coletados

In [8]:
first_house=imoveis_centro[0]

In [9]:
first_house.find('h6',  class_='tm-property__title').text.strip() #título

'Água Branca'

In [10]:
first_house.find('h6', class_='tm-property__price').text.strip().replace("R$ ","").replace(".","") #valor

'700000,00'

In [11]:
first_house.find('div', class_='tm-property__excerpt').text.strip() #descrição

'Descrição do Apto:\r\n63m² de área útil\r\n114m² de área total\r\n02 Dormitórios, sendo 1 suíte\r\n02 Vagas\r\n01 Banheiro...'

In [12]:
first_house.find('span', class_='tm-property__attributes-area').text.strip() #área em m2

'63 m²'

In [13]:
first_house.find('span', class_='tm-property__attributes-bedrooms').text.strip() #quartos

'2'

In [14]:
first_house.find('span', class_='tm-property__attributes-bathrooms').text.strip() #banheiros

'1'

In [15]:
first_house.find('span', class_='tm-property__attributes-parking-spot').text.strip() #vagas

'2'

In [16]:
first_house.find('div', class_='tm-property__code').text.strip()

'Cód. 249'

# Função para coletar os dados - sem paginação

In [17]:
names = []
values = []
descriptions = []
bedrooms = []
bathrooms = []
parking_spots = []
area = []
codigo = []

for container in imoveis_centro: 
    
    name = container.find('h6',  class_='tm-property__title').text.strip()
    names.append(name)    
    
    value = float(container.find('h6', class_='tm-property__price').text.strip().replace("R$ ","").replace(".","").replace(",","."))
    values.append(value) 
    
    description = container.find('div', class_='tm-property__excerpt').text.strip()
    descriptions.append(description)    
        
    cod = int(container.find('div', class_='tm-property__code').text.strip().replace("Cód. ",""))
    codigo.append(cod)
    
    if container.find('div', class_='tm-property__attributes') is not None:
        if container.find('span', class_='tm-property__attributes-bedrooms') is not None:
            bedroom =  int(container.find('span', class_='tm-property__attributes-bedrooms').text.strip())
            bedrooms.append(bedroom)
        else:
            bedroom = 0
            bedrooms.append(bedroom)
        
        if container.find('span', class_='tm-property__attributes-bathrooms') is not None:
            bathroom =  int(container.find('span', class_='tm-property__attributes-bathrooms').text.strip())
            bathrooms.append(bathroom)
        else:
            bathroom = 0
            bathrooms.append(bathroom) 
        
        if container.find('span', class_='tm-property__attributes-parking-spot') is not None:
            parking_spot =  int(container.find('span', class_='tm-property__attributes-parking-spot').text.strip())
            parking_spots.append(parking_spot)
        else:
            parking_spot = 0
            parking_spots.append(parking_spot)
        
        if container.find('span', class_='tm-property__attributes-area') is not None:
            area2 =  float(container.find('span', class_='tm-property__attributes-area').text.strip().replace(" m²",""))
            area.append(area2)
        else:
            area2 = 0
            area.append(area2)       
       
  

# Criando DataFrame

In [18]:
df = pd.DataFrame({'nome':names,
                   'valor':values, 
                   'descricao': descriptions, 
                   'quartos': bedrooms,
                   'banheiros':bathrooms,
                   'vagas':parking_spots,
                   'area':area,
                   'codigo':codigo
                  })


In [19]:
df.head(15)

,nome,valor,descricao,quartos,banheiros,vagas,area,codigo
0,Água Branca,700000.0,Descrição do Apto:\r\n63m² de área útil\r\n114...,2,1,2,63.0,249
1,Água Branca,905000.0,Varanda Gourmet e Lazer completo,3,0,0,0.0,252
2,Barra Funda,340000.0,"Apartamento com dois dormitórios, sala, cozinh...",2,0,0,86.0,26
3,Barra Funda,530000.0,,2,1,0,66.0,34
4,Bela Vista,200000.0,Apartamento tipo studio na avenida 9 de julho ...,0,0,0,0.0,421
5,Bela Vista,255000.0,vende-se studio mobiliado ao lado da avenida p...,1,0,0,30.0,272
6,Bela Vista,290000.0,Belo apartamento situado na região da Bela Vis...,1,0,0,39.0,67
7,Bela Vista,300000.0,W. 158,2,0,0,39.0,158
8,Bela Vista,320000.0,Temos para você uma ótima oportunidade de apê ...,0,0,1,28.0,48
9,Bela Vista,357000.0,Apartamento Kitnet - Bela Vista \r\n\r\nAo lad...,0,0,0,46.0,265


In [20]:
df.dtypes

nome          object
valor        float64
descricao     object
quartos        int64
banheiros      int64
vagas          int64
area         float64
codigo         int64
dtype: object

# PAGINANDO

In [21]:
paginas = [str(i) for i in range(1,6)]
paginas

['1', '2', '3', '4', '5']

### Testando o endereço

In [22]:
for page in paginas:
    
    response = get('http://www.imoveisnocentrosp.com.br/buscar?page=' + page + '&venda=Sim')
    print(response, page, 'http://www.imoveisnocentrosp.com.br/buscar?page=' + page + '&venda=Sim')

<Response [200]> 1 http://www.imoveisnocentrosp.com.br/buscar?page=1&venda=Sim
<Response [200]> 2 http://www.imoveisnocentrosp.com.br/buscar?page=2&venda=Sim
<Response [200]> 3 http://www.imoveisnocentrosp.com.br/buscar?page=3&venda=Sim
<Response [200]> 4 http://www.imoveisnocentrosp.com.br/buscar?page=4&venda=Sim
<Response [200]> 5 http://www.imoveisnocentrosp.com.br/buscar?page=5&venda=Sim


### Função para coleta das páginas

In [23]:
names = []
values = []
descriptions = []
bedrooms = []
bathrooms = []
parking_spots = []
area = []
codigo = []

for page in paginas:
    
    response = get('http://www.imoveisnocentrosp.com.br/buscar?page=' + page + '&venda=Sim')
    
    sleep(randint(10,30))
     
    page_html = BeautifulSoup(response.content, 'html.parser')
    
    imoveis_centro = page_html.find('div', class_='tm-property__wrap').find_all('div',  class_='tm-property__item')
    
    for container in imoveis_centro: 
        name = container.find('h6',  class_='tm-property__title').text.strip()
        names.append(name)    
    
        value = float(container.find('h6', class_='tm-property__price').text.strip().replace("R$ ","").replace(".","").replace(",","."))
        values.append(value) 
    
        description = container.find('div', class_='tm-property__excerpt').text.strip()
        descriptions.append(description)    
        
        cod = int(container.find('div', class_='tm-property__code').text.strip().replace("Cód. ",""))
        codigo.append(cod) 
    
        if container.find('div', class_='tm-property__attributes') is not None:
            
            if container.find('span', class_='tm-property__attributes-bedrooms') is not None:
                bedroom =  int(container.find('span', class_='tm-property__attributes-bedrooms').text.strip())
                bedrooms.append(bedroom)
            else:
                bedroom = 0
                bedrooms.append(bedroom)
        
            if container.find('span', class_='tm-property__attributes-bathrooms') is not None:
                bathroom =  int(container.find('span', class_='tm-property__attributes-bathrooms').text.strip())
                bathrooms.append(bathroom)
            else:
                bathroom = 0
                bathrooms.append(bathroom) 
        
            if container.find('span', class_='tm-property__attributes-parking-spot') is not None:
                parking_spot =  int(container.find('span', class_='tm-property__attributes-parking-spot').text.strip())
                parking_spots.append(parking_spot)
            else:
                parking_spot = 0
                parking_spots.append(parking_spot)

            if container.find('span', class_='tm-property__attributes-area') is not None:
                area2 =  float(container.find('span', class_='tm-property__attributes-area').text.strip().replace(" m²",""))
                area.append(area2)
            else:
                area2 = 0
                area.append(area2)

    print(response, page, 'http://www.imoveisnocentrosp.com.br/buscar?page=' + page + '&venda=Sim')    
           

<Response [200]> 1 http://www.imoveisnocentrosp.com.br/buscar?page=1&venda=Sim
<Response [200]> 2 http://www.imoveisnocentrosp.com.br/buscar?page=2&venda=Sim
<Response [200]> 3 http://www.imoveisnocentrosp.com.br/buscar?page=3&venda=Sim
<Response [200]> 4 http://www.imoveisnocentrosp.com.br/buscar?page=4&venda=Sim
<Response [200]> 5 http://www.imoveisnocentrosp.com.br/buscar?page=5&venda=Sim


# Criando DataFrame paginado

In [24]:
df_paginado = pd.DataFrame({'nome':names,
                   'valor':values, 
                   'descricao': descriptions, 
                   'quartos': bedrooms,
                   'banheiros':bathrooms,
                   'vagas':parking_spots,
                   'area':area,
                   'codigo':codigo
                  })

In [25]:
df_paginado.head(75)

,nome,valor,descricao,quartos,banheiros,vagas,area,codigo
0,Água Branca,700000.0,Descrição do Apto:\r\n63m² de área útil\r\n114...,2,1,2,63.0,249
1,Água Branca,905000.0,Varanda Gourmet e Lazer completo,3,0,0,0.0,252
2,Barra Funda,340000.0,"Apartamento com dois dormitórios, sala, cozinh...",2,0,0,86.0,26
3,Barra Funda,530000.0,,2,1,0,66.0,34
4,Bela Vista,200000.0,Apartamento tipo studio na avenida 9 de julho ...,0,0,0,0.0,421
5,Bela Vista,255000.0,vende-se studio mobiliado ao lado da avenida p...,1,0,0,30.0,272
6,Bela Vista,290000.0,Belo apartamento situado na região da Bela Vis...,1,0,0,39.0,67
7,Bela Vista,300000.0,W. 158,2,0,0,39.0,158
8,Bela Vista,320000.0,Temos para você uma ótima oportunidade de apê ...,0,0,1,28.0,48
9,Bela Vista,357000.0,Apartamento Kitnet - Bela Vista \r\n\r\nAo lad...,0,0,0,46.0,265


In [26]:
df_paginado.to_csv('df_paginado-imoveisnocentro.csv',index=False)